# Using Selenium, Beatutiful Soup & Requests: _Hershel, O My Bag & Whimsy and Row_

In this section we will be using selenium and beautiful soup to scrape all of the women's bags from Hershel, O My Bag & Whimsy and Row. We will get the: picture,price, product link, and description.

Resources used to help in this process can be found in other notebooks.

#### Import your needed Libraries:

In [310]:
import time
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import cv2
import numpy as np 
import pandas as pd
import re
from PIL import Image
import requests
from io import BytesIO
from scraping_functions import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Create variables for each of the URL

We will do this for each of the pages we will be pulling from on _Hershel, O My Bag & Whimsy and Row_.

In [2]:
hershel_backpacks = 'https://herschel.com/shop/womens/backpacks'
hershel_fp = 'https://herschel.com/shop/womens/hip-packs'
hershel_totes = 'https://herschel.com/shop/womens/totes'
hershel_messenger = 'https://herschel.com/shop/womens/messengers'

O_my_bag = 'https://www.omybag.nl/product-category/women/women-bags/'

#only the first 13-ish bags
WR_bags = 'https://whimsyandrow.com/collections/bags-clutches-pouches?sort_by=title-ascending'

### O My Bag:

Now that we have our page variables set up, we can scrape the bags. After inspecting on the website it appears we can just use beautiful soup for this task.

In [5]:
omb_page = requests.get(O_my_bag)
omb_soup = BeautifulSoup(omb_page.content, 'lxml')
divs_imgs = omb_soup.findAll('div', {'class':'content-product'})

In [13]:
omb_links = omb_soup.findAll('a', {'class':'woocommerce-LoopProduct-link'})
omb_href = [x['href'] for x in omb_links]

Now let's grab the images:

In [25]:
omb_images_tags = omb_soup.findAll('img', {'class':'attachment-woocommerce_thumbnail'})
omb_images = [x['src'] for x in omb_images_tags]

Now, let's get into the details like price and name:

In [27]:
omb_prod_details = omb_soup.findAll('div', {'class':'product-details'})

In [38]:
omb_names = [x.find('div', {'class': 'product-title'}).text.strip() for x in omb_prod_details]
omb_prices = [x.find('span', {'class': 'price'}).text.strip() for x in omb_prod_details]

### Create a DataFrame and use our function to download pictures :

Below we will use the lists created above and make a dataframe. Then we will define a function to download the images into our directory, and save the image file name in a new column of the DataFrame.

In [57]:
table = [omb_names,omb_prices,omb_href,omb_images]
OMB_df = pd.DataFrame(table).transpose()

In [58]:
OMB_df.rename( columns={0:'product_name', 1: 'product_price',2:'URL',3:'img_url'},inplace=True)

In [59]:
OMB_df.head(3)

,product_name,product_price,URL,img_url
0,Lola – Black Croco – Soft Grain / Classic Leather,€169.00,https://www.omybag.nl/shop/women/lola-black-so...,https://static1.omybag.nl/wp-content/uploads/2...
1,Jean Backpack – Black – Soft Grain Leather,€299.00,https://www.omybag.nl/shop/women/jean-backpack...,https://www.omybag.nl/wp-content/uploads/2019/...
2,Mila (short handle) – Natural – Classic Leather,€329.00,https://www.omybag.nl/shop/men/mila-short-hand...,https://static2.omybag.nl/wp-content/uploads/2...


#### Save all images using an apply

In [65]:
c = 0
OMB_df['img'] = OMB_df.img_url.apply(lambda x: save_src_image_apply('../../data/','omybag_',x))

Saved omybag__1.png
Saved omybag__2.png
Saved omybag__3.png
Saved omybag__4.png
Saved omybag__5.png
Saved omybag__6.png
Saved omybag__7.png
Saved omybag__8.png
Saved omybag__9.png
Saved omybag__10.png
Saved omybag__11.png
Saved omybag__12.png
Saved omybag__13.png
Saved omybag__14.png
Saved omybag__15.png
Saved omybag__16.png
Saved omybag__17.png
Saved omybag__18.png
Saved omybag__19.png
Saved omybag__20.png
Saved omybag__21.png
Saved omybag__22.png
Saved omybag__23.png
Saved omybag__24.png
Saved omybag__25.png
Saved omybag__26.png
Saved omybag__27.png
Saved omybag__28.png
Saved omybag__29.png
Saved omybag__30.png
Saved omybag__31.png
Saved omybag__32.png
Saved omybag__33.png
Saved omybag__34.png
Saved omybag__35.png
Saved omybag__36.png
Saved omybag__37.png
Saved omybag__38.png
Saved omybag__39.png
Saved omybag__40.png
Saved omybag__41.png
Saved omybag__42.png
Saved omybag__43.png
Saved omybag__44.png
Saved omybag__45.png
Saved omybag__46.png
Saved omybag__47.png
Saved omybag__48.png
S

In [67]:
OMB_df.to_csv('../../omb_df.csv')

### Whimsy and Row:

In [73]:
wr_page = requests.get(WR_bags)
wr_soup = BeautifulSoup(wr_page.content, 'lxml')
divs_wr = wr_soup.findAll('div', {'class':'product-frame'})

In [106]:
wr_url = ['https://whimsyandrow.com' + x.find('a')['href'] for x in divs_wr[:-3]]
wr_names = [x.find('a')['title'] for x in divs_wr[:-3]]
wr_images_url = ['https:'+x.find('img')['src'] for x in divs_wr[:-3]] 

Now, let's get the prices which are in a different div:

In [97]:
divs_wr_2 = wr_soup.findAll('div', {'class':'product-title'})
prices = [x.find('h6').text for x in divs_wr_2[:-3]]

### Create this whimsy and Row DF and save the images:

In [107]:
table = [wr_names,prices,wr_url,wr_images_url]
wr_df = pd.DataFrame(table).transpose()
wr_df.rename(columns={0:'product_name', 1: 'product_price',2:'URL',3:'img_url'},inplace=True)

#### Save all images using an apply:

In [112]:
c = 0
wr_df['img'] = wr_df.img_url.apply(lambda x: save_src_image_apply('../../data/','whimsy_row_',x))

Saved whimsy_row__1.png
Saved whimsy_row__2.png
Saved whimsy_row__3.png
Saved whimsy_row__4.png
Saved whimsy_row__5.png
Saved whimsy_row__6.png
Saved whimsy_row__7.png
Saved whimsy_row__8.png
Saved whimsy_row__9.png
Saved whimsy_row__10.png
Saved whimsy_row__11.png
Saved whimsy_row__12.png
Saved whimsy_row__13.png


In [113]:
wr_df.to_csv('../../whimsy_row_df.csv')

### _Hershel_:

In [115]:
h_page_1 = requests.get(hershel_backpacks)
h_soup_1 = BeautifulSoup(h_page_1.content, 'lxml')
divs_h = h_soup_1.findAll('div', {'class':'col-lg-4 col-md-6 col-xs-12 m-b-2'})

In [235]:
#three of the bookbags do not have multiple colors
colors_carousel = [x.findAll('div',{'class':'carousel'}) if x.findAll('div',{'class':'carousel'}) 
                   else x.find('div',{'class':'product-image'}) for x in divs_h]

h_bb_imgs = [i[0].findAll('img',{'class':'swatch'}) if str(type(i)) == "<class 'bs4.element.ResultSet'>" 
             else [i.find('img')] for i in colors_carousel]

In [304]:
h_bookbag_colors = [[i['data-color'] if 'data-color' in str(i) else 'NA' for i in x] for x in h_bb_imgs]
h_bookbag_img = [['https://herschel.com'+i['data-product-image'] if 'data-product-image' in 
                  str(i) else 'https://herschel.com'+i['data-src'] for i in x] for x in h_bb_imgs]

In [212]:
names = [x.find('div',{'class':'col-xs-6 title-line'}).text for x in divs_h]

In [320]:
prices = [x.find('span',{'class':'hsco-set-currency'}).text for x in divs_h]
prod_h_url = [x.find('a',{'class':'js-product-grid-link'})['href'] for x in divs_h]

In [307]:
names_adj = []
for i in range(len(names)):
    n = len(h_bb_imgs[i])
    names_adj.extend([names[i]]*n)

In [322]:
prices_adj =  []
for i in range(len(prices)):
    n = len(h_bb_imgs[i])
    prices_adj.extend([prices[i]]*n)

In [328]:
prod_h_url_adj =  []
for i in range(len(prod_h_url)):
    n = len(h_bb_imgs[i])
    prod_h_url_adj.extend([prod_h_url[i]]*n)

In [216]:
len(h_bookbag_img),len(names)

(94, 98)

In [329]:
len(names_adj),len(prices_adj),len(prod_h_url_adj)

(1403, 1403, 1403)

Let's flatten the lists of colors and images:

In [336]:
h_bookbag_colors_fl = [color for sublist in h_bookbag_colors for color in sublist]
h_bookbag_img_fl = [img for sublist in h_bookbag_img for img in sublist]
# len(h_bookbag_colors_fl),len(h_bookbag_img_fl)

Make the DataFrame:

In [338]:
table = [names_adj,prices_adj,h_bookbag_colors_fl,prod_h_url_adj,h_bookbag_img_fl]
herschel_df = pd.DataFrame(table).transpose()
herschel_df.rename(columns={0:'product_name', 1: 'product_price',2:'color',3:'URL',4:'img_url'},inplace=True)

#### Save all images using an apply:

In [341]:
c = 0
herschel_df['img'] = herschel_df.img_url.apply(lambda x: save_src_image_apply('../../data/','herschel',x))

Saved herschel_1.png
Saved herschel_2.png
Saved herschel_3.png
Saved herschel_4.png
Saved herschel_5.png
Saved herschel_6.png
Saved herschel_7.png
Saved herschel_8.png
Saved herschel_9.png
Saved herschel_10.png
Saved herschel_11.png
Saved herschel_12.png
Saved herschel_13.png
Saved herschel_14.png
Saved herschel_15.png
Saved herschel_16.png
Saved herschel_17.png
Saved herschel_18.png
Saved herschel_19.png
Saved herschel_20.png
Saved herschel_21.png
Saved herschel_22.png
Saved herschel_23.png
Saved herschel_24.png
Saved herschel_25.png
Saved herschel_26.png
Saved herschel_27.png
Saved herschel_28.png
Saved herschel_29.png
Saved herschel_30.png
Saved herschel_31.png
Saved herschel_32.png
Saved herschel_33.png
Saved herschel_34.png
Saved herschel_35.png
Saved herschel_36.png
Saved herschel_37.png
Saved herschel_38.png
Saved herschel_39.png
Saved herschel_40.png
Saved herschel_41.png
Saved herschel_42.png
Saved herschel_43.png
Saved herschel_44.png
Saved herschel_45.png
Saved herschel_46.p

Saved herschel_361.png
Saved herschel_362.png
Saved herschel_363.png
Saved herschel_364.png
Saved herschel_365.png
Saved herschel_366.png
Saved herschel_367.png
Saved herschel_368.png
Saved herschel_369.png
Saved herschel_370.png
Saved herschel_371.png
Saved herschel_372.png
Saved herschel_373.png
Saved herschel_374.png
Saved herschel_375.png
Saved herschel_376.png
Saved herschel_377.png
Saved herschel_378.png
Saved herschel_379.png
Saved herschel_380.png
Saved herschel_381.png
Saved herschel_382.png
Saved herschel_383.png
Saved herschel_384.png
Saved herschel_385.png
Saved herschel_386.png
Saved herschel_387.png
Saved herschel_388.png
Saved herschel_389.png
Saved herschel_390.png
Saved herschel_391.png
Saved herschel_392.png
Saved herschel_393.png
Saved herschel_394.png
Saved herschel_395.png
Saved herschel_396.png
Saved herschel_397.png
Saved herschel_398.png
Saved herschel_399.png
Saved herschel_400.png
Saved herschel_401.png
Saved herschel_402.png
Saved herschel_403.png
Saved hersc

Saved herschel_717.png
Saved herschel_718.png
Saved herschel_719.png
Saved herschel_720.png
Saved herschel_721.png
Saved herschel_722.png
Saved herschel_723.png
Saved herschel_724.png
Saved herschel_725.png
Saved herschel_726.png
Saved herschel_727.png
Saved herschel_728.png
Saved herschel_729.png
Saved herschel_730.png
Saved herschel_731.png
Saved herschel_732.png
Saved herschel_733.png
Saved herschel_734.png
Saved herschel_735.png
Saved herschel_736.png
Saved herschel_737.png
Saved herschel_738.png
Saved herschel_739.png
Saved herschel_740.png
Saved herschel_741.png
Saved herschel_742.png
Saved herschel_743.png
Saved herschel_744.png
Saved herschel_745.png
Saved herschel_746.png
Saved herschel_747.png
Saved herschel_748.png
Saved herschel_749.png
Saved herschel_750.png
Saved herschel_751.png
Saved herschel_752.png
Saved herschel_753.png
Saved herschel_754.png
Saved herschel_755.png
Saved herschel_756.png
Saved herschel_757.png
Saved herschel_758.png
Saved herschel_759.png
Saved hersc

Saved herschel_1070.png
Saved herschel_1071.png
Saved herschel_1072.png
Saved herschel_1073.png
Saved herschel_1074.png
Saved herschel_1075.png
Saved herschel_1076.png
Saved herschel_1077.png
Saved herschel_1078.png
Saved herschel_1079.png
Saved herschel_1080.png
Saved herschel_1081.png
Saved herschel_1082.png
Saved herschel_1083.png
Saved herschel_1084.png
Saved herschel_1085.png
Saved herschel_1086.png
Saved herschel_1087.png
Saved herschel_1088.png
Saved herschel_1089.png
Saved herschel_1090.png
Saved herschel_1091.png
Saved herschel_1092.png
Saved herschel_1093.png
Saved herschel_1094.png
Saved herschel_1095.png
Saved herschel_1096.png
Saved herschel_1097.png
Saved herschel_1098.png
Saved herschel_1099.png
Saved herschel_1100.png
Saved herschel_1101.png
Saved herschel_1102.png
Saved herschel_1103.png
Saved herschel_1104.png
Saved herschel_1105.png
Saved herschel_1106.png
Saved herschel_1107.png
Saved herschel_1108.png
Saved herschel_1109.png
Saved herschel_1110.png
Saved herschel_1

In [342]:
herschel_df.to_csv('../../herschel_df.csv')

### Herschel Fannie Packs:

In [343]:
h_page_1 = requests.get(hershel_fp)
h_soup_1 = BeautifulSoup(h_page_1.content, 'lxml')
divs_h = h_soup_1.findAll('div', {'class':'col-lg-4 col-md-6 col-xs-12 m-b-2'})

In [385]:
colors_carousel_fp = [x.findAll('div',{'class':'carousel'}) if x.findAll('div',{'class':'carousel'}) 
                   else x.find('div',{'class':'product-image'}) for x in divs_h]
h_fp_imgs = [i[0].findAll('img',{'class':'swatch'}) if str(type(i)) == "<class 'bs4.element.ResultSet'>" 
             else [i.find('img')] for i in colors_carousel_fp]

In [386]:
h_fannie_colors = [[i['data-color'] if 'data-color' in str(i) else 'NA' for i in x] for x in h_fp_imgs]
h_fannie_img = [['https://herschel.com'+i['data-product-image'] if 'data-product-image' in 
                  str(i) else 'https://herschel.com'+i['data-src'] for i in x] for x in h_fp_imgs]
names_fp = [x.find('div',{'class':'col-xs-6 title-line'}).text for x in divs_h]
prices_fp = [x.find('span',{'class':'hsco-set-currency'}).text for x in divs_h]
prod_h_url_fp = ['https://herschel.com'+x.find('a',{'class':'js-product-grid-link'})['href'] for x in divs_h]

Extend prices, names, and product url lists to ensure they have the same lengths as all of the colors.

In [387]:
names_adj_fp = []
for i in range(len(names_fp)):
    n = len(h_bb_imgs_fp[i])
    names_adj_fp.extend([names_fp[i]]*n)
    
prices_adj_fp =  []
for i in range(len(prices_fp)):
    n = len(h_fp_imgs[i])
    prices_adj_fp.extend([prices_fp[i]]*n)

prod_h_url_adj_fp =  []
for i in range(len(prod_h_url_fp)):
    n = len(h_fp_imgs[i])
    prod_h_url_adj_fp.extend([prod_h_url_fp[i]]*n)

Flatten the images and colors lists:

In [391]:
h_fp_colors_fl = [color for sublist in h_fannie_colors for color in sublist]
h_fp_img_fl = [img for sublist in h_fannie_img for img in sublist]
len(h_fp_colors_fl),len(h_fp_img_fl),len(prod_h_url_adj_fp)

(282, 282, 282)

Create the DataFrame of the Herschel Fannie Packs:

In [392]:
table = [names_adj_fp,prices_adj_fp,h_fp_colors_fl,prod_h_url_adj_fp,h_fp_img_fl]
herschel_df_fp = pd.DataFrame(table).transpose()
herschel_df_fp.rename(columns={0:'product_name', 1: 'product_price',2:'color',3:'URL',4:'img_url'},inplace=True)

In [393]:
herschel_df_fp.head()

,product_name,product_price,color,URL,img_url
0,Fourteen Hip Pack,24.99,Black,https://herschel.com/shop/hip-packs/fourteen-h...,https://herschel.com/content/dam/herschel/prod...
1,Fourteen Hip Pack,24.99,Black Crosshatch,https://herschel.com/shop/hip-packs/fourteen-h...,https://herschel.com/content/dam/herschel/prod...
2,Fourteen Hip Pack,24.99,Ash Rose,https://herschel.com/shop/hip-packs/fourteen-h...,https://herschel.com/content/dam/herschel/prod...
3,Fourteen Hip Pack,24.99,Raven Crosshatch,https://herschel.com/shop/hip-packs/fourteen-h...,https://herschel.com/content/dam/herschel/prod...
4,Fourteen Hip Pack,24.99,Purple Velvet,https://herschel.com/shop/hip-packs/fourteen-h...,https://herschel.com/content/dam/herschel/prod...


#### Save all images using an apply:

In [405]:
c = 0
herschel_df_fp['img'] = herschel_df_fp.img_url.apply(lambda x: save_src_image_apply('../../data/','herschel_fp',x))

Saved herschel_fp_1.png
Saved herschel_fp_2.png


KeyboardInterrupt: 

### Herschel Totes:

In [395]:
h_page_1 = requests.get(hershel_totes)
h_soup_1 = BeautifulSoup(h_page_1.content, 'lxml')
divs_h = h_soup_1.findAll('div', {'class':'col-lg-4 col-md-6 col-xs-12 m-b-2'})

In [396]:
colors_carousel_t = [x.findAll('div',{'class':'carousel'}) if x.findAll('div',{'class':'carousel'}) 
                   else x.find('div',{'class':'product-image'}) for x in divs_h]
h_t_imgs = [i[0].findAll('img',{'class':'swatch'}) if str(type(i)) == "<class 'bs4.element.ResultSet'>" 
             else [i.find('img')] for i in colors_carousel_t]

In [397]:
h_tote_colors = [[i['data-color'] if 'data-color' in str(i) else 'NA' for i in x] for x in h_t_imgs]
h_tote_img = [['https://herschel.com'+i['data-product-image'] if 'data-product-image' in 
                  str(i) else 'https://herschel.com'+i['data-src'] for i in x] for x in h_t_imgs]
names_t = [x.find('div',{'class':'col-xs-6 title-line'}).text for x in divs_h]
prices_t = [x.find('span',{'class':'hsco-set-currency'}).text for x in divs_h]
prod_h_url_t = ['https://herschel.com'+x.find('a',{'class':'js-product-grid-link'})['href'] for x in divs_h]

Extend prices, names, and product url lists to ensure they have the same lengths as all of the colors.

In [398]:
names_adj_t = []
for i in range(len(names_t)):
    n = len(h_t_imgs[i])
    names_adj_t.extend([names_t[i]]*n)
    
prices_adj_t =  []
for i in range(len(prices_t)):
    n = len(h_t_imgs[i])
    prices_adj_t.extend([prices_t[i]]*n)
1
prod_h_url_adj_t =  []
for i in range(len(prod_h_url_t)):
    n = len(h_t_imgs[i])
    prod_h_url_adj_t.extend([prod_h_url_t[i]]*n)

Flatten the images and colors lists:

In [400]:
h_t_colors_fl = [color for sublist in h_tote_colors for color in sublist]
h_t_img_fl = [img for sublist in h_tote_img for img in sublist]
len(h_t_colors_fl),len(h_t_img_fl),len(prod_h_url_adj_t)

(162, 162, 162)

Create the DataFrame of the Herschel Fannie Packs:

In [401]:
table = [names_adj_t,prices_adj_t,h_t_colors_fl,prod_h_url_adj_t,h_t_img_fl]
herschel_df_t = pd.DataFrame(table).transpose()
herschel_df_t.rename(columns={0:'product_name', 1: 'product_price',2:'color',3:'URL',4:'img_url'},inplace=True)

In [402]:
herschel_df_t.head()

,product_name,product_price,color,URL,img_url
0,Bamfield Tote | Mid-Volume,64.99,Black,https://herschel.com/shop/totes/bamfield-tote-...,https://herschel.com/content/dam/herschel/prod...
1,Bamfield Tote | Mid-Volume,64.99,Frog Camo,https://herschel.com/shop/totes/bamfield-tote-...,https://herschel.com/content/dam/herschel/prod...
2,Bamfield Tote | Mid-Volume,64.99,Fall Floral - Hoffman Collection,https://herschel.com/shop/totes/bamfield-tote-...,https://herschel.com/content/dam/herschel/prod...
3,Bamfield Tote | Mid-Volume,64.99,Ash Rose,https://herschel.com/shop/totes/bamfield-tote-...,https://herschel.com/content/dam/herschel/prod...
4,Bamfield Tote | Mid-Volume,64.99,Light Grey Crosshatch,https://herschel.com/shop/totes/bamfield-tote-...,https://herschel.com/content/dam/herschel/prod...


#### Save all images using an apply:

In [404]:
c = 0
herschel_df_t['img'] = herschel_df_t.img_url.apply(lambda x: save_src_image_apply('../../data/','herschel_t',x))

Saved herschel_t_1.png
Saved herschel_t_2.png
Saved herschel_t_3.png
Saved herschel_t_4.png
Saved herschel_t_5.png
Saved herschel_t_6.png
Saved herschel_t_7.png
Saved herschel_t_8.png
Saved herschel_t_9.png
Saved herschel_t_10.png
Saved herschel_t_11.png
Saved herschel_t_12.png
Saved herschel_t_13.png
Saved herschel_t_14.png
Saved herschel_t_15.png
Saved herschel_t_16.png
Saved herschel_t_17.png
Saved herschel_t_18.png
Saved herschel_t_19.png
Saved herschel_t_20.png
Saved herschel_t_21.png
Saved herschel_t_22.png
Saved herschel_t_23.png
Saved herschel_t_24.png
Saved herschel_t_25.png
Saved herschel_t_26.png
Saved herschel_t_27.png
Saved herschel_t_28.png
Saved herschel_t_29.png
Saved herschel_t_30.png
Saved herschel_t_31.png
Saved herschel_t_32.png
Saved herschel_t_33.png
Saved herschel_t_34.png
Saved herschel_t_35.png
Saved herschel_t_36.png
Saved herschel_t_37.png
Saved herschel_t_38.png
Saved herschel_t_39.png
Saved herschel_t_40.png
Saved herschel_t_41.png
Saved herschel_t_42.png
S